In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# BART data's excel into a DF
rawBART_data = 'Resources/FY Avg Wkdy Exits by Station_6.xlsx'
rawBART = pd.read_excel(rawBART_data, header=3)
# Using only Name, 2012, 2013, and 2014 data
dfBART = rawBART[['Station','FY11', 'FY12', 'FY13', 'FY14']].copy()
# Using only downtown SF stations' rider exits
dfEXITS = dfBART.iloc[25:29]
dfEXITS

,Station,FY11,FY12,FY13,FY14
25,Embarcadero,35603.036,37699.788,41058.692308,42546.2
26,Montgomery Street,33710.808,36517.092,39166.773279,40864.1
27,Powell Street,25138.656,26400.488,28294.546559,28321.6
28,Civic Center,18119.396,18739.088,20341.599190,20810.4


In [3]:
# Get the sum of all downtown stations by year
dtsf_2011 = np.round(dfEXITS['FY11'].sum(), 0)
dtsf_2012 = np.round(dfEXITS['FY12'].sum(), 0)
dtsf_2013 = np.round(dfEXITS['FY13'].sum(), 0)
dtsf_2014 = np.round(dfEXITS['FY14'].sum(), 0)
print(f"2011 riders: {dtsf_2011}\n\n2012 riders: {dtsf_2012}\n\n2013 riders: {dtsf_2013}\n\n2014 riders: {dtsf_2014}")

2011 riders: 112572.0

2012 riders: 119356.0

2013 riders: 128862.0

2014 riders: 132542.0


In [4]:
# SF salaries data into a DF
rawSALARY_data = 'Resources/Salaries.csv'
rawSALARY = pd.read_csv(rawSALARY_data)
# Drop 2011's data
rawSALARY = rawSALARY[rawSALARY.Year != 2011]
# Using only BasePay, Benefits, TotalPay, and Year
dfSALARY = rawSALARY[['BasePay', 'Benefits', 'TotalPay', 'Year']].copy()
dfSALARY.head()

/Users/michelcaraco/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,BasePay,Benefits,TotalPay,Year
36159,128809,44430.1,362844.66,2012
36160,302578,69810.2,321552.11,2012
36161,111921,53102.3,336393.73,2012
36162,296943,72047.9,314759.60,2012
36163,126863,44438.2,337204.86,2012


In [5]:
# Drop NaNs from dfSALARY as new DF
cleaned_salary = dfSALARY.dropna(axis=0)
# cleaned_salary = dfSALARY
# Drop rows if they contain a non-numeric value (eg: "Not Provided")
cleaned = cleaned_salary.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()
# cleaned = cleaned_salary
# Average with 2 decimals by year
cleaned = np.round(cleaned.groupby('Year').mean(), 2)
cleaned

,BasePay,Benefits,TotalPay
Year,,,
2012,65436.41,26439.97,74113.26
2013,69630.03,24131.70,78808.29
2014,66564.42,24789.60,75471.84


In [6]:
# Insert the rider exit counts for 1 combined DF
cleaned.insert(3, "Riders", [dtsf_2012, dtsf_2013, dtsf_2014])
cleaned

,BasePay,Benefits,TotalPay,Riders
Year,,,,
2012,65436.41,26439.97,74113.26,119356.0
2013,69630.03,24131.70,78808.29,128862.0
2014,66564.42,24789.60,75471.84,132542.0


In [7]:
cleaned.index.names = ["Years"]
cleaned

,BasePay,Benefits,TotalPay,Riders
Years,,,,
2012,65436.41,26439.97,74113.26,119356.0
2013,69630.03,24131.70,78808.29,128862.0
2014,66564.42,24789.60,75471.84,132542.0


In [8]:
rds_connection_string = "root:necrid@127.0.0.1/RidershipData"
#mysql+pymysql instead of mysql
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [9]:
engine.table_names()

InternalError: (pymysql.err.InternalError) (1049, "Unknown database 'ridershipdata'") (Background on this error at: http://sqlalche.me/e/2j85)

In [ ]:
# engine = create_engine('sqlite://', echo = False)
# conn = engine.connect()

cleaned.to_sql(name = 'cleaned', con = engine, if_exists = 'append', index = True)

# engine.execute("SELECT * FROM cleaned").fetchall()

In [ ]:
pd.read_sql_query('select * from cleaned', con = engine).head()